# Change the color of data points in a JSON file

With this script you can change colours of data points in existing MeshView-compatible JSON files. 

There are two ways to change the RGB colors:
1. Specify the colours in a separate excel file, e.g. 'colors.xlsx'.
This excel file should have the following structure:
    + column "fileName" : In this column the name of the files are defined (without the extension)
    + column "r" : the red part of the RGB colour [0-255]
    + column "g" : the green part of the RGB colour [0-255]
    + column "b" : the blue part of the RGB colour [0-255] 
    
2. Use the same RGB colour for all files you want to change. When asked the question "What is the RGB code you want to use (e.g. 255 0 0):" Fill in `255 0 0`, without brackets, but with space between the r, g, and b number.

The json file will be overwritten with using the same file name (in the original folder)

**Note:** If there are multiple sets of data points within one file, all of them will get the same color. If you want to give each set a different color, then you are better of changing the color first and creating a new custom cloud.

In [1]:
import json
import os
import pandas as pd
from pathlib import Path


## Define the folder the files you want to change are stored in

In [2]:
#  If files are located in a specific folder, or in a folder that is not part of the current working directory, define it here.
folder_name = input("What is the folder name/file path the files are stored in? ")

# Find the file names in the folder:
fnames = [fname.__str__() for fname in sorted(Path(folder_name).rglob('*.json'))]

## Define the color you want to replace the current color with


In [12]:
# Define the color of the data in the json file
answer = input("Are colours defined in a separate file? Yes (y) or No (n): ")

if answer == "n": 
    RGB = input("What is the RGB code you want to use (e.g. 255 0 0): ") 
    colors = {}
    colors["r"] = int(RGB.split(" ")[0])
    colors["g"] = int(RGB.split(" ")[1])
    colors["b"] = int(RGB.split(" ")[2])
    expNames = None
elif answer == "y":
    colour_file = input("What is the file name in which the colours are defined for each json file you want to change? ")
    colors = pd.read_excel(colour_file + ".xlsx")
    expNames = colors.fileName.to_list()

## Function to update the color

In [14]:
def update_color(fname, answer, colors, expNames=None):
    exp_name = os.path.splitext(os.path.basename(fname))[0]
    if expNames is not None:
        r, g, b = (int(colors.r[i]) if answer == "y" and exp_name == str(expNames[i])
                   else int(colors["r"]) for i in range(len(expNames)))
    else:
        r = int(colors["r"])
        g = int(colors["g"])
        b = int(colors["b"])

    with open(fname, 'r') as f:
        file = json.load(f)
        for ii in range(len(file)):
            file[ii]["r"] = r
            file[ii]["g"] = g
            file[ii]["b"] = b

    with open(fname, 'w', encoding='utf-8') as f:
        f.write(json.dumps(file, ensure_ascii=False, indent=4))


## Change the color of all files using the function above

In [ ]:
# Loop over all files and change the color
for fname in fnames:
    update_color(fname, answer, colors, expNames)
